* 과적합은 학습데이터를 불필요할 정도로 과하게 암기하여 훈련데이터에 포함된 노이즈까지 학습한 상태라고 해석할 수 있음
* 이러한 과적합을 막기 위해 다양한 방법이 있음

### 1. 데이터 양 늘리기
* 데이터 양이 적을 경우, 과적합이 발생하기 쉬움
* 그렇기 때문에 데이터 양을 늘릴 수록 모델은 데이터의 일반적인 패턴을 하습하여 과적합 방지 가능

----
* 만약 데이터 양이 적을 경우, 데이터 증강(Data Augmentation)을 통해 데이터 양을 늘림
* 텍스트 데이터의 경우, 번역후 재번역을 통해 새로운 데이터를 만들어내는 역번역(Back Translation) 등의 방법이 있음

### 2. 모델의 복잡도 줄이기
* 과적합 현상이 발견되었을때, 모델에서 할 수 있는 조치는 은닉층의 수나 매개변수의 수를 줄일 수 있음
* 인공신경망에서는 **모델에 있는 매개변수들의 수를 모델의 수용력(capacity)**이라고 하기도 함


### 3. 가중치 규제(Regularization) 적용
* 적은 수의 매개변수로 된 간단한 모델을 만들기 위한 방법으로 가중치 규제(Regularization)가 있음
    * L1 규제 : 가중치 $w$들의 절대값 합계를 비용함수에 추가. L1 노름이라고도 함
    * L2 규제 : 모든 가중치 $w$들의 제곱합을 비용함수에 추가. L2 노름이라고도 함
    
* L1 규제는 기존 비용함수에 모든 가중치에 대해 $\lambda \mid w \mid$를 더한 값을 비용함수로 하고,
* L2 규제는 기존의 비용함수에 모든 가중치에 대해 $\frac{1}{2} \lambda w^2$를 더한 값을 비용함수로 함
* $\lambda$는 규제의 강도를 정하는 하이파라미터로, $\lambda$가 크면 모델이 훈련데이터에 대해 적합한 매개변수를 찾는 것보다 규제를 위해 추가한 항들을 작게 유지하는 것을 우선한다는 것을 의미

* L1 규제는 어떤 특성들이 모델에 영향을 주고 있는지 판단하고자 할 때 유용하지만, 이런 판단이 필요없을 경우 L2규제가 더 잘 작동함
* 인공신경망에서 L2 규제는 가중치 감쇠(weight decay)라고도 부름

### 4. 드롭아웃(Dropout)
* 학습과정에서 신경망의 일부를 사용하지 않는 방법
* 예를 들어 드롭아웃 비율을 0.5로 한다면, 학습과정마다 랜덤으로 절반의 뉴런을 사용하지 않고, 절반의 뉴런만을 사용함
* 일반적으로 학습에서만 사용하고, 예측시에는 사용하지 않음
* 이는 학습에서 인공신경망이 특정 뉴런 또는 특정 조합에 너무 의존적이게 되는 것을 방지해 주고, 매번 랜덤선택으로 서로 다른 신경망들을 앙상블하여 사용하는 것과 같은 효과를 내어 과적합를 방지함

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
max_words = 10000
num_classes = 46

model = Sequential()
model.add(Dense(256, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

### 5. 기울기 소실(Gradient Vanishing) 와 기울기 폭주(Gradient Exploding)

* 기울기 소실 : 역전파 과정에서 갈수록 기울기가 점차적으로 작아지는 현상
* 기울기 폭주 : 반대로 기울기가 점차 커지면서 가중치들이 비정상적으로 큰 값으로 발산하는 현상, RNN에서 쉽게 발생
---
#### 예방 방법
#### 1) ReLU와 ReLU의 변형
* 시그모이드 함수는 0과 1에 가까울수록 미분값이 0에 가까워지므로 기울기가 점차 사라져 역전파시 문제 발생
* 이를 완화하는 방법이 활성화함수로 시그모이드 함수나 하이퍼볼릭탄젠트 함수 대신에 ReLU 또는 ReLU 변형 함수를 사용하는 것임

    * 은닉층에서는 시그모이니 함수 사용 금지!!
    * Leaky ReLU를 사용하면 모든 입력값에서 기울기가 0이 되지 않아 죽은 ReLU 문제 해결
    * 따라서 은닉층에서는 ReLU나 Leaky ReLU 등 변형함수 사용하기 
    
#### 2) 그래디언트 클리핑(Gradient Clipping)
* 말그대로 기울기 값을 자르는 것을 의미
* 기울기 폭주를 막기 위해 임계값을 넘지 않도록 자름
* 즉, 임계치만큼 크기를 감소시키는 것. 이는 RNN에서 유용

In [3]:
from tensorflow.keras import optimizers
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)

#### 3) 가중치 초기화

* 가중치 초기값에 따라 훈련 결과가 달라지는데 가중치 초기화만 잘 해줘도 기울기 소실문제를 완화할 수 있음

##### 3-1) 세이비어 초기화(Xavier Initialization)
* 균등분포와 정규분포로 구분할 수 있고, 이전층의 뉴런수와 다음층의 뉴런수를 가지고 식을 만듦

$$W \sim Uniform(-\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }}, +\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }})$$

* 이는 $\sqrt{\frac{6}{ {n}_{in} + {n}_{out} }}$를 $m$이라고 하였을때, -m 과 +m 사이의 균등분포를 의미
* 정규분포로 초기화할 경우 평균이 0이고 표준편차 σ가 다음을 만족하도록 함
$$σ=\sqrt{\frac { 2 }{ { n }_{ in }+{ n }_{ out } } }$$

----
* 세이비어 초기화는 여러층의 기울기 분산을 균형 맞춰 특정층이 너무 튀는 것을 방지함
* 그러나 시그모이드 함수나 하이퍼볼릭탄젠트 함수와 같은 S자 형태인 활성화 함수와 함께 사용할 경우 좋은 성능을 보이지만, ReLU와 함께 사용할 경우 성능이 좋아지지 않음
* ReLU 와 같은 함수를 사용할 때에는 **He 초기화(He initialization)** 사용

##### 3-2) He 초기화(He initialization)

* 마찬가지로 정규분포와 균등분포 두가지 방법이 있으나, 세이비어 초기화와 달리 다음 층의 뉴런수를 반영하지 않음
* 이전 층의 뉴런수만을 고려

$$W\sim Uniform(- \sqrt{\frac { 6 }{ { n }_{ in } } } , \space\space + \sqrt{\frac { 6 }{ { n }_{ in } } } )$$

$$σ=\sqrt{\frac { 2 }{ { n }_{ in } } }$$

---
* 시그모이드 함수나 하이퍼볼릭탄젠트 함수를 사용할 경우에는 세이비어 초기화 효율적
* ReLU 계열 함수의 경우 He가 효율적


#### 4) 배치 정규화
* 배치 정규화는 인공신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만듬

#### 4-1) 내부공변량 변화(Internal Covariate Shift)
* 내부공변량 변화 : 학습과정에서 층별로 입력데이터 분포가 달라지는 현상
* 이전 층의 학습에 의해 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력데이터의 분포가 달라짐
* 즉, 딥러닝 모델의 불안정성은 층마다 입력의 분포가 달라지기 때문이라고 주장.... 어찌됐든 정규화는 도움이 됨

---
* 공변량 변화는 훈련데이터의 분포와 테스트 데이터의 분포가 다른 경우
* 내부 공변량 변화는 신경망 층 간 입력데이터의 분포 변화

#### 4-2) 배치 정규화(Batch Normalization)
* 한번에 들어오는 배치 단위로 정규화하는 것
* 각 층에서 활성화함수를 통과하기 전에 실시
* 입력에 대해 평균을 0으로 만들고 정규화함
* 그리고 정규화된 데이터에 대해 스케일과 시프트를 수행
* β는 시프트를 할 때 사용하며 다음 레이어에 일정한 범위의 값들만 전달

Input: 미니배치 $B = \{{x}^{(1)}, {x}^{(2)}, ..., {x}^{(m)}\}$

Output: $y^{(i)} = BN_{γ, β}(x^{(i)})$

$$μ_{B} ← \frac{1}{m} \sum_{i=1}^{m} x^{(i)} \text{ # 미니 배치에 대한 평균 계산}$$

$$σ^{2}_{B} ← \frac{1}{m} \sum_{i=1}^{m} (x^{(i)} - μ_{B})^{2}\text{ # 미니 배치에 대한 분산 계산}$$
$$\hat{x}^{(i)} ← \frac{x^{(i)} - μ_{B}}{\sqrt{σ^{2}_{B}+ε}}\text{ # 정규화}$$
$$y^{(i)} ← γ\hat{x}^{(i)} + β = BN_{γ, β}(x^{(i)}) \text{ # 스케일 조정(γ)과 시프트(β)를 통한 선형 연산}$$



* $m$은 미니 배치에 있는 샘플의 수
* $μ_{B}$는 미니 배치 $B$에 대한 평균.
* $σ_{B}$는 미니 배치 $B$에 대한 표준편차.
* $\hat{x}^{(i)}$은 평균이 0이고 정규화 된 입력 데이터.
* $ε$은 가 0일 때, 분모가 0이 되는 것을 막는 작은 양수. 보편적으로 $10^{-5}$
* $γ$는 정규화 된 데이터에 대한 스케일 매개변수로 학습 대상
* $β$는 정규화 된 데이터에 대한 시프트 매개변수로 학습 대상
* $y^{(i)}$는 스케일과 시프트를 통해 조정한 의 최종 결과

**배치 정규화는 학습 시 배치 단위의 평균과 분산들을 차례대로 받아 이동 평균과 이동 분산을 저장해놓았다가 테스트 할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화를 합니다.**

---
* 시그모이드 함수나 하이퍼볼릭탄젠트 함수에서도 기울기 소실 문제 크게 개선
* 가중치 초기화 덜 민감
* 훨씬 큰 학습률을 사용할 수 있어 속도 개선
* 과적합 효과 방지와 dropout 유사한 효과
* 계산과정이 늘어나는 것이므로 예측시 실행 시간이 길어짐
* 효과는 굉장함
----

#### 4-3) 배치정규화의 한계
* 미니배치 크기에 의존적 : 너무 작으면 안됨
* RNN 적용 어려움 : RNN은 각 시점(time step)마다 다른 통계치를 가져 배치 정규화가 어려움
* 따라서 RNN에는 **층 정규화(layer normalization)** 방법 적용


#### 5) 층 정규화(Layer Normalization)

https://www.google.co.kr/search?tbm=bks&hl=ko&q=%EB%B0%B0%EC%B9%98+%EC%A0%95%EA%B7%9C%ED%99%94
https://mlexplained.com/2018/11/30/an-overview-of-normalization-methods-in-deep-learning/
https://m.blog.naver.com/laonple/220808903260
http://blog.naver.com/PostView.nhn?blogId=sogangori&logNo=220551548382
https://www.quora.com/What-are-the-practical-differences-between-batch-normalization-and-layer-normalization-in-deep-neural-networks
https://theneuralperspective.wordpress.com/2016/10/27/gradient-topics/